# LTV Analysis - Exploratory data analysis
Author: Ricardo Raspini Motta

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import calibration_curve 
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
import lifelines

from sklearn.model_selection import GridSearchCV

In [2]:
base = catalog.load("base")
xs = catalog.load("xs")

[11/26/23 18:50:34] INFO     Loading data from 'base' (CSVDataSet)...                           ]8;id=709174;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=603254;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    ERROR    _request non-retriable exception: Invalid Credentials, 401                ]8;id=158021;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/gcsfs/retry.py\retry.py]8;;\:]8;id=916741;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/gcsfs/retry.py#157\157]8;;\
                             ╭────────────────── Traceback (most recent call last) ──────────────────╮             
                             │ /home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-pa │             
                             │ ckages/gcsfs/retry.py:123 in retry_request                            │             
                             │                                                                       │             
                             │   120 │   │   try:                                                    │             
                             │   121 │   │   │   if retry > 0:                                       │             
                             │   122 │   │   │   │   await asyncio.sleep(min(random.random() + 2 **  │             
                             │ ❱ 123 │   │   │   return await func(*args, **kwargs)                  │             
                             │   124 │   │   except (                                                │             
                             │   125 │   │   │   HttpError,                                          │             
                             │   126 │   │   │   requests.exceptions.RequestException,               │             
                             │                                                                       │             
                             │ /home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-pa │             
                             │ ckages/gcsfs/core.py:430 in _request                                  │             
                             │                                                                       │             
                             │    427 │   │   │   info = r.request_info  # for debug only            │             
                             │    428 │   │   │   contents = await r.read()                          │             
                             │    429 │   │   │                                                      │             
                             │ ❱  430 │   │   │   validate_response(status, contents, path, args)    │             
                             │    431 │   │   │   return status, headers, info, contents             │             
                             │    432 │                                                              │             
                             │    433 │   async def _call(                                           │             
                             │                                                                       │             
                             │ /home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-pa │             
                             │ ckages/gcsfs/retry.py:110 in validate_response                        │             
                             │                                                                       │             
                             │   107 │   │   elif "invalid" in str(msg):                             │             
                             │   108 │   │   │   raise ValueError(f"Bad Request: {path}\n{msg}")     │             
                             │   109 │   │   elif error:                                             │             
                             │ ❱ 110 │   │   │   raise HttpError(error)                              │             
                             │   111 │   │   elif status:                                            │             
                             │   112 │   │  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/core.py:193 │
│ in load                                                                                          │
│                                                                                                  │
│   190 │   │   self._logger.debug("Loading %s", str(self))                                        │
│   191 │   │                                                                                      │
│   192 │   │   try:                                                                               │
│ ❱ 193 │   │   │   return self._load()                                                            │
│   194 │   │   except DatasetError:                                                               │
│   195 │   │   │   raise                                                                          │
│   196 │   │   except Exception as exc:                                                           │
│                                                                                                  │
│ /home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/extras/dataset │
│ s/pandas/csv_dataset.py:163 in _load                                                             │
│                                                                                                  │
│   160 │   │   │   return pd.read_csv(load_path, **self._load_args)                               │
│   161 │   │                                                                                      │
│   162 │   │   load_path = f"{self._protocol}{PROTOCOL_DELIMITER}{load_path}"                     │
│ ❱ 163 │   │   return pd.read_csv(                                                                │
│   164 │   │   │   load_path, storage_options=self._storage_options, **self._load_args            │
│   165 │   │   )                                                                                  │
│   166                                                                                            │
│                                                                                                  │
│ /home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/pandas/io/parsers/re │
│ aders.py:948 in read_csv                                                                         │
│                                                                                                  │
│    945 │   )                                                                                     │
│    946 │   kwds.update(kwds_defaults)                                                            │
│    947 │                                                                                         │
│ ❱  948 │   return _read(filepath_or_buffer, kwds)                                                │
│    949                                                                                           │
│    950                                                                                           │
│    951 # iterator=True -> TextFileReader                                                         │
│                                                                                                  │
│ /home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/pandas/io/parsers/re │
│ aders.py:611 in _read                                                                            │
│                                                                                                  │
│    608 │   _validate_names(kwds.get("names", None))                                              │
│    609 │                                                                                         │
│    610 │   # Create the parser.                            

### Base table

In [ ]:
base

In [ ]:
base["age_bucket"].value_counts()

In [ ]:
base["product"].value_counts()

In [ ]:
base["channel"].value_counts()

In [ ]:
base["commission"].describe()

In [ ]:
base["operating_system"].value_counts()

In [ ]:
base.loc[:, "operating_system"] = base["operating_system"].replace({"iPadOS": "iOS", "iPhone OS": "iOS"})

### Cross selling table

In [ ]:
xs

#### All users are unique in the base table

In [ ]:
len(base["user_id"].unique()) == len(base)

#### That is not the same at the Cross Selling table

In [ ]:
len(xs["user_id"].unique()) == len(xs)

#### Looking for the most frequent client

In [ ]:
xs["user_id"].value_counts().head(1)

In [ ]:
base.loc[base["user_id"]=="995528dadd"]

In [ ]:
xs.loc[xs["user_id"]=="995528dadd"]

## Commission per product

In [ ]:
def get_commission_per_product(df):
    comission_per_product = df.groupby("product").agg({"commission": ["sum","count"]})
    comission_per_product.columns = list(map('_'.join, comission_per_product.columns.values))
    
    comission_per_product.loc[:,"commission_per_unit"] = comission_per_product["commission_sum"] / comission_per_product["commission_count"]
    return comission_per_product

In [ ]:
comission_per_product_base = get_commission_per_product(base)
comission_per_product_xs = get_commission_per_product(xs)
comission_per_product = pd.concat([
        comission_per_product_base,
        comission_per_product_xs
    ],
     ignore_index=False).reset_index()

#### The table below shows that the commision per unit is far bigger in the cross selling products.
That said, the cross selling brings a big financial opportunity

In [ ]:
comission_per_product

In [ ]:
xs = pd.merge(
    xs,
    pd.get_dummies(xs["product"]),
    left_index=True,
    right_index=True,
)

grouped_xs = xs.groupby("user_id").sum()[["commission", "product_x", "product_y"]].reset_index()

df = pd.merge(
    base,
    grouped_xs,
    on="user_id",
    how="left",
    suffixes=["_base", "_xs"]
)

datetime_cols = ["customer_churned_at", "customer_started_at"]
for col in datetime_cols:
    df.loc[:, col] = pd.to_datetime(df[col], errors="coerce", utc=False)

df.loc[:,"is_churn"] = ~df["customer_churned_at"].isna()

In [ ]:
df.loc[:,"is_xs"] = np.where(df[["product_x", "product_x"]].fillna(0).sum(axis=1) >0, 1, 0)

#### Churn rates in relation with categorical values

In [ ]:
df.groupby("product").agg({"is_churn": "mean"})

In [ ]:
df.groupby("channel").agg({"is_churn": "mean"})

In [ ]:
df.groupby("operating_system").agg({"is_churn": "mean"})

In [ ]:
df.groupby("age_bucket").agg({"is_churn": "mean"})

In [ ]:
df.groupby("is_xs").agg({"is_churn": "mean"})

In [ ]:
df.loc[:, "customer_started_at"] = pd.to_datetime(df["customer_started_at"])

df = df.set_index("customer_started_at")

plot_users = df.groupby(pd.Grouper(freq="M")).agg(
    {
        "user_id" : "count",
        "is_xs": "sum",
        "product_x": "sum",
        "product_y": "sum",
    }
)

## Sales per time

In [ ]:
sns.set_style("whitegrid")
palette = sns.color_palette()
fig,axs = plt.subplots(1,2, figsize = (15,6))
sns.lineplot(plot_users["user_id"], ax = axs[0], color = palette[0], label="Base Sales")
sns.lineplot(plot_users["product_x"] + plot_users["product_y"], ax = axs[1], color = palette[1], label="Cross sales")
plt.suptitle("Evolution of sales, base and cross", fontsize=18)
plt.tight_layout()
plt.show()

## Churn related to numerical variables

In [ ]:
inf_lim, sup_lim = 0,40
col_to_plot = "commission_base"
dataplot = df.loc[(df[col_to_plot]<=sup_lim) & (df[col_to_plot]>=inf_lim)]
sns.kdeplot(data = dataplot, x= col_to_plot, hue=dataplot["is_churn"].astype(int))
plt.title("Distribution of churn in commissions")
plt.show()

In [ ]:
df.loc[:, "count_xs"] = df["product_x"].fillna(0) + df["product_y"].fillna(0)
inf_lim, sup_lim = 0,2
col_to_plot = "count_xs"
dataplot = df.loc[(df[col_to_plot]<=sup_lim) & (df[col_to_plot]>=inf_lim)]
sns.histplot(data = dataplot, x= col_to_plot, hue=dataplot["is_churn"].astype(int))
plt.title("Distribution of churn in cross selling customers")
plt.show()

In [ ]:
df = df.reset_index()


In [ ]:
df.loc[df["is_churn"], "days_to_churn"] = pd.to_datetime(df.loc[df["is_churn"],"customer_churned_at"]).subtract(df.loc[df["is_churn"] ,"customer_started_at"]).dt.days

In [ ]:
sns.histplot(df.loc[df["is_churn"], "days_to_churn"])

In [ ]:
users_churn_negative = df.loc[df.loc[:, "days_to_churn"] <0]["user_id"].to_list()

In [ ]:
base.loc[base["user_id"].isin(users_churn_negative)]